In [ ]:
import FollowUpAnalysis_readData as fud
import FollowUpAnalysis_Plot as fup
import pandas as pd
import os
current_dir = os.getcwd()
working_dir = os.path.dirname(current_dir)

# analysis by usmle test

## load data

In [ ]:
analysis_version_dir, modelname = f'{working_dir}/Data/Data2402', 'checkpoints2'
# analysis_version_dir, modelname  = f'{working_dir}/Data/Data2312', 'checkpoints1'
# analysis_version_dir, modelname  = f'{working_dir}/Data/Data2311', 'checkpoints0'

follow = fud.read_followup_json_files_to_df_byusmletest(f'{analysis_version_dir}/{modelname}_annotated_jsons.json')
aggdf = pd.read_csv(f'{analysis_version_dir}/5voteaggdf.csv')
category_order = ['All True', 'Mostly True', 'Mostly False', 'All False']
aggdf['performance'] = pd.Categorical(aggdf['performance'], 
                                            categories=category_order, 
                                            ordered=True)

df = pd.merge(aggdf, follow, left_on= 'question', right_on='usmle_test', how='right')
print(df.shape)
df.head()

## plot

In [ ]:
follow_up_quality = df.loc[:,('question',"basic_knowledge_count_of_false",
                                               'interpretation_and_association_count_of_false',
                                               'total_count_of_false','performance'
                                                          )]
# change colname for ploting
follow_up_quality.columns = ['question',"wrong in basic_kno",'wrong in interp_&_assoc',
                             'wrong in total','performance']


fup.plot_hist_performance_bycat(follow_up_quality,'performance','wrong in total',70,8)


In [ ]:
fup.plot_hist_performance_bycat(follow_up_quality,'performance','wrong in basic_kno',120,8)

In [ ]:
fup.plot_hist_performance_bycat(follow_up_quality,'performance','wrong in interp_&_assoc',120,8)

# analysis by followup

## loaddata

In [ ]:
filelist = [f'{working_dir}/Data/Data2311/checkpoints0_annotated_jsons.json',
            f'{working_dir}/Data/Data2312/checkpoints1_annotated_jsons.json',
            f'{analysis_version_dir}/checkpoints2_annotated_jsons.json']

filename = [filname.split('/')[-1].split('_annotated_jsons')[0] for filname in filelist]

dflist = [fud.read_followup_json_files_to_df_byfollowupq(file) for file in filelist]

aggdflist = [fud.summarize_performance(df,'classification') for df in dflist]
# print(aggdflist[0].head())
diagdflist= [fud.summarize_performance(df,'involve_diagnosis') for df in dflist]
# print(diagdflist[0].head())
keydflist = [fud.summarize_performance(df,'key') for df in dflist]
# print(keydflist[0].head())

In [ ]:
concatenateddf = pd.concat([df['percentage_of_true'].rename(filename) for df, filename in zip(keydflist, filename)], axis=1)
concatenateddf.head()

In [ ]:
concatenateddf = pd.concat([df['percentage_of_true'].rename(filename) for df, filename in zip(diagdflist, filename)], axis=1)
concatenateddf.head()

In [ ]:
concatenateddf = pd.concat([df['percentage_of_true'].rename(filename) for df, filename in zip(aggdflist, filename)], axis=1)
concatenateddf.head()

In [ ]:
concatenateddf = pd.concat([df[['count_of_all','percentage_of_false','count_of_false']].rename(columns={
                                                                                        'count_of_all': f'{filename}_count_of_all',
                                                                                        'percentage_of_false': f'{filename}_percentage_of_false',
                                                                                        'count_of_false': f'{filename}_count_of_false'
                                                                                            }) for df, filename in zip(aggdflist, filename)], axis=1)
concatenateddf.to_csv(f'{analysis_version_dir}/classification_compare_df_falserate.csv')
concatenateddf.head()

## plot

In [ ]:
for aggdf in aggdflist:
    fup.plot_hist_for_followup_perform_byclass(aggdf,'count_of_all','category')

In [ ]:
for aggdf in aggdflist:
    fup.plot_hist_for_followup_perform_byclass(aggdf,'count_of_false','category')

In [ ]:
for aggdf in aggdflist:
    fup.plot_hist_for_followup_perform_byclass(aggdf,'percentage_of_false','category',percentage=True)